# MIMIC-IV-on-FHIR Google Healthcare API Tutorial
This tutorial will walk through using MIMIC-IV-on-FHIR on GCP. The GCP Healthcare API provides the primary features all FHIR servers have.

The following features will be explored:
- Search
  - Search by gender
  - Search by condition
  - Search by medication
- Export
  - patient-everything 
  - subsets
- Aggregation?

In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import google.auth
from google.auth.transport import requests
from google.cloud import storage
import json
import time

In [ ]:
load_dotenv(load_dotenv(Path(Path.cwd()).parents[0].parents[0] / '.env'))

In [ ]:
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_TOPIC = os.getenv('GCP_TOPIC')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_BUCKET = os.getenv('GCP_BUCKET')
GCP_DATASET = os.getenv('GCP_DATASET')
GCP_FHIRSTORE = os.getenv('GCP_FHIRSTORE')
GCP_EXPORT_FOLDER = os.getenv('GCP_EXPORT_FOLDER')
GCP_TOPIC_PATIENT_EVERYTHING = os.getenv('GCP_TOPIC_PATIENT_EVERYTHING')

credentials, project = google.auth.default()

In [ ]:
GCP_PROJECT

In [ ]:
session = requests.AuthorizedSession(credentials)
base_url = "https://healthcare.googleapis.com/v1"

project_url = f'{base_url}/projects/{GCP_PROJECT}/locations/{GCP_LOCATION}'
fhir_url = f'{project_url}/datasets/{GCP_DATASET}/fhirStores/{GCP_FHIRSTORE}/fhir'
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}



## Search

### Search By Gender

In [ ]:
resource_type = 'Patient'
gender = 'female'

resource_url = f'{fhir_url}/{resource_type}/_search?gender={gender}'
response = session.post(resource_url, headers=headers)
response.raise_for_status()
resources = response.json()
resources

## Export

In [ ]:
# Support functions
def get_resource_ids(fhir_url, resource_type):
    resource_url = f'{fhir_url}/{resource_type}/_search?_elements=id'
    response = session.post(resource_url, headers=headers)
    response.raise_for_status()
    resources = response.json()
    patient_ids = [ entry['resource']['id'] for entry in resources['entry']]
    return patient_ids

### Patient-everything

In [ ]:
GCP_PATIENT_EVERYTHING_FOLDER = f'patient-everything/bundles-{time.strftime("%Y%m%d-%H%M%S")}'

In [ ]:
def send_patient_everything(export_url, headers, patient_id, page_num=1):
    response = session.get(export_url, headers=headers)
    response.raise_for_status()
    resp_fhir = response.json()

    if 'error' in resp_fhir:
        print('ERROR IN RESPONSE')
    elif resp_fhir['resourceType'] == 'OperationOutcome':
        print(resp_fhir['issue'][0])
    elif  ((resp_fhir['resourceType'] == 'Bundle') and ('link' in resp_fhir)):
        filename = export_bundle_to_storage(resp_fhir, patient_id, page_num)
        print(f'Stored file: {filename}')
        link_info = [
            resp for resp in resp_fhir['link'] if resp['relation'] == 'next'
        ]
        if len(link_info) > 0:
            send_patient_everything(
                link_info[0]['url'], headers, patient_id, page_num + 1
            )
    else:
        filename = export_bundle_to_storage(resp_fhir, patient_id, page_num)
        print(f'Stored file: {filename}')


    return resources

def export_bundle_to_storage(resp_fhir, patient_id, page_num):
    bundle = resp_fhir

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(GCP_BUCKET)
    filename = f"{GCP_PATIENT_EVERYTHING_FOLDER}/patient-{patient_id}-page{page_num}"
    blob = bucket.blob(filename)
    blob.upload_from_string(json.dumps(bundle))
    return filename

In [ ]:
# Get some patient_ids

resource_type = 'Patient'
output_resource_types = 'Patient,Encounter,Condition,Procedure' # resource types to output
num_patients = 1
count = 100 # how many resources per bundle page

patient_list = get_resource_ids(fhir_url, resource_type)
patient_list

if num_patients > len(patient_list):
    num_patients = len(patient_list)
for idx in range(0,num_patients):
    patient_id = patient_list[idx]
    export_url = f'{fhir_url}/Patient/{patient_id}/$everything?_count={count}&_type={output_resource_types}'
    print(export_url)
    resources = send_patient_everything(export_url, headers, patient_id)